In [1]:
## Importing required Libraries
import os
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
## Get working directory
PATH = os.getcwd()

In [3]:
## Path to save the embedding and checkpoints generated
LOG_DIR = PATH + '/embedded/'

In [8]:
## Load data
df = pd.read_csv("features.csv",index_col =0)

In [13]:
import csv
open("output.tsv", "w").write("\n".join("\t".join(e.strip() for e in i) for i in csv.reader(open("label.csv"))))


23643

In [14]:
## Load the metadata file. Metadata consists your labels. This is optional. Metadata helps us visualize(color) different clusters that form t-SNE
metadata = os.path.join(LOG_DIR, 'label.tsv')

In [15]:
# Generating PCA and 
pca = PCA(n_components=50,
         random_state = 123,
         svd_solver = 'auto'
         )
df_pca = pd.DataFrame(pca.fit_transform(df))
df_pca = df_pca.values

In [16]:
## TensorFlow Variable from data
tf_data = tf.Variable(df_pca)

In [17]:
## Running TensorFlow Session
with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf_data.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()

In [18]:
# One can add multiple embeddings.
embedding = config.embeddings.add()
embedding.tensor_name = tf_data.name
# Link this tensor to its metadata(Labels) file
embedding.metadata_path = metadata
# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

In [ ]:
#tensorboard --logdir embedded/